# ライブラリのインポート

In [47]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets
%matplotlib inline

<br>

# 同時変換行列の定義

In [48]:
def Hx(theta):
    """
    x軸周りに回転する同時変換行列を求める
    
    Parameters
    ----------
    theta : float
        回転角度(deg)

    Returns
    -------
    Hx : numpy.ndarray
        x軸周りにthetaだけ回転させる同時変換行列
    """
    return np.array([[1,             0,              0, 0],
                     [0, np.cos(theta), -np.sin(theta), 0],
                     [0, np.sin(theta),  np.cos(theta), 0],
                     [0,             0,              0, 1]])

def Hy(theta):
    """
    y軸周りに回転する同時変換行列を求める
    
    Parameters
    ----------
    theta : float
        回転角度(deg)

    Returns
    -------
    Hy : numpy.ndarray
        y軸周りにthetaだけ回転させる同時変換行列
    """
    return np.array([[ np.cos(theta), 0,  np.sin(theta), 0],
                     [             0, 1,              0, 0],
                     [-np.sin(theta), 0,  np.cos(theta), 0],
                     [0,              0,              0, 1]])

def Hz(theta):
    """
    z軸周りに回転する同時変換行列を求める
    
    Parameters
    ----------
    theta : float
        回転角度(deg)

    Returns
    -------
    zx : numpy.ndarray
        z軸周りにthetaだけ回転させる同時変換行列
    """
    return np.array([[np.cos(theta),  -np.sin(theta), 0, 0],
                     [np.sin(theta),   np.cos(theta), 0, 0],
                     [            0,               0, 1, 0],
                     [            0,               0, 0, 1]])

def Hp(x, y, z):
    """
    (x, y, z)方向に並進移動する同時変換行列を求める
    
    Parameters
    ----------
    x : float
        x方向の移動量
    y : float
        y方向の移動量
    z : float
        z方向の移動量

    Returns
    -------
    Hp : numpy.ndarray
        (x, y, z)方向に並進移動させる同時変換行列
    """
    return np.array([[1, 0, 0, x],
                     [0, 1, 0, y],
                     [0, 0, 1, z],
                     [0, 0, 0, 1]])

<br>

# DH法による同時変換行列

In [53]:
def Hdh(theta, d, alpha, a):
    """
    DH法を求める
    
    Parameters
    ----------
    theta : float
        リンクの回転角度(deg)
    d : float
        リンク間のオフセット
    alpha : float
        リンクのねじれ角
    a : float
        リンク長

    Returns
    -------
    Hdh_array : numpy.ndarray
        DH法に基づいた同時変換行列
    """
    #Hdh_array = Hz(theta)@Hp(0, 0, d)@Hx(alpha)@Hp(a, 0, 0) # DH法　
    Hdh_array = Hx(alpha)@Hp(a, 0, 0)@Hz(theta)@Hp(0, 0, d) # 修正DH法
    return Hdh_array


# 描画関連の定義

In [78]:
def draw_link_coordinate(ax, Hdh):   
    """
    Hdh : ワールド座標系原点から，N番目のリンク座標原点までの同時変換行列(4×4)
    """
    # n番目のlinkの座標系のx方向の単位ベクトル
    link_x = Hdh@np.array([[1],
                           [0],
                           [0],
                           [1]])
    #print(link_x[0][0]**2+link_x[1][0]**2+link_x[2][0]**2)

    # n番目のlinkの座標系のy方向の単位ベクトル
    link_y = Hdh@np.array([[0],
                           [1],
                           [0],
                           [1]])

    # n番目のlinkの座標系のz方向の単位ベクトル
    link_z = Hdh@np.array([[0],
                           [0],
                           [1],
                           [1]])                        

    # ワールド座標系を原点(0,0,0)とした時の，n番目のlinkの原点座標
    x = Hdh[0][3]; y = Hdh[1][3]; z = Hdh[2][3]
    
    # n番目のlinkの座標系をplot (x方向)
    ax.plot([x, link_x[0][0]], [y, link_x[1][0]], [z, link_x[2][0]], "o-", color="red", ms=2) 
    # n番目のlinkの座標系をplot (y方向)
    ax.plot([x, link_y[0][0]], [y, link_y[1][0]], [z, link_y[2][0]], "o-", color="green", ms=2) 
    # n番目のlinkの座標系をplot (z方向)
    ax.plot([x, link_z[0][0]], [y, link_z[1][0]], [z, link_z[2][0]], "o-", color="blue", ms=2)

In [162]:
def generate_vbox_text_widget(num):
    """
    text widgetsをnum個作成 -> Vboxに格納して縦に並べる
    
    Parameters
    ----------
    num : int
        ロボットのリンクの数

    Returns
    -------
    vox_text_widgets : ipywidgets.widgets.widget_box.VBox
        text widgetsをnum個，縦に並べたVBox
    """
    text_widgets = []
    for i in range(num):
      text_widgets.append(ipywidgets.FloatText())
    vox_text_widgets = ipywidgets.VBox(text_widgets)
    return vox_text_widgets

def generate_vbox_slider_widget(num):
    """
    slider widgetsをnum個作成 -> Vboxに格納して縦に並べる
    
    Parameters
    ----------
    num : int
        ロボットのリンクの数

    Returns
    -------
    vox_slider_widgets : ipywidgets.widgets.widget_box.VBox
        slider widgetsをnum個，縦に並べたVBox
    """
    slider_widgets = []
    for i in range(num):
      slider_widgets.append(ipywidgets.FloatSlider(value=0.0, min=-180.0, max=180.0))
    vox_slider_widgets = ipywidgets.VBox(slider_widgets)
    return vox_slider_widgets


def link_slider_and_text(slider_widgets_name, text_widgets_name, num):
    """
    Boxに格納されている複数のslider widet と text widget を連携させる
    
    Parameters
    ----------
    slider_widgets_name : ipywidgets.widgets.interaction.interactive
        widgetsの名前
    text_widgets_name : ipywidgets.widgets.widget_box.VBox
        widgetsの名前
    num : int
        linkの数

    """
    for i in range(num):
      ipywidgets.link((slider_widgets_name.children[i], 'value'), (text_widgets_name.children[i], 'value'))

def draw(num):
    # interactive widget (slider widget)を作成　(main文の引数と連携)
    #posture_sliders = ipywidgets.interactive(main, theta1=(-180, 180), theta2=(-180, 180), theta3=(-180, 180), theta4=(-180, 180), theta5=(-180, 180), theta6=(-180, 180))
    posture_sliders = generate_vbox_slider_widget(num)
    # text widgetを作成
    posture_texts = generate_vbox_text_widget(num)

    # slider widget と　posture widget を横に並べる
    slider_and_text = ipywidgets.Box([posture_sliders, posture_texts])

    # slider wiget と text widget を連携
    link_slider_and_text(posture_sliders, posture_texts, num)

    # リセットボタン
    reset_button = ipywidgets.Button(description = "Reset")
    # 姿勢のリセットボタン
    def reset_values(button):
        posture_sliders.children[0].value = 0.0
        posture_sliders.children[1].value = 0.0
        posture_sliders.children[2].value = 0.0
        posture_sliders.children[3].value = 0.0
        posture_sliders.children[4].value = 0.0
        posture_sliders.children[5].value = 0.0
    reset_button.on_click(reset_values)

    params = {}
    params["param"] = posture_sliders
    #print(params)
    final_widgets = ipywidgets.interactive(kkkk, params=params)
    
    display(slider_and_text)

<br>

# 実行

In [163]:
def kkkk(params):
    print(params)
    # 描画関連 -----------------------------------
    fig = plt.figure(figsize=(5,10))
    ax1 = fig.add_subplot(2,1,1, projection='3d')
    
    # ここからDH法の実装 ------------------------
    l1 = 4.0; l2 = 4.0; l3 = 2.0
    theta1 = params.children[0].value
    theta2 = params.children[1].value
    theta3 = params.children[2].value
    theta4 = params.children[3].value
    theta5 = params.children[4].value
    theta6 = params.children[5].value

    # DH法による同時変換行列 0->1
    theta1 = np.radians(theta1); d1 = 0; alpha1 = -np.pi/2; a1 = 0
    Hdh01 = Hdh(theta1, d1, alpha1, a1) 
    # DH法による同時変換行列 1->2
    theta2 = np.radians(theta2); d2 = 0; alpha2 =0; a2 = l1
    Hdh12 = Hdh(theta2, d2, alpha2, a2) 
    # DH法による同時変換行列 2->3
    theta3 = np.radians(theta3); d3 = 0; alpha3 = np.pi/2; a3 = 0
    Hdh23 = Hdh(theta3, d3, alpha3, a3) 
    # DH法による同時変換行列 3->4
    theta4 = np.radians(theta4); d4 = l2; alpha4 = -np.pi/2; a4 = 0
    Hdh34 = Hdh(theta4, d4, alpha4, a4) 
    # DH法による同時変換行列 4->5
    theta5 = np.radians(theta5); d5 = 0; alpha5 = np.pi/2; a5 = 0
    Hdh45 = Hdh(theta5, d5, alpha5, a5) 
    # DH法による同時変換行列 5->6
    theta6 = np.radians(theta6); d6 = l3; alpha6 = 0; a6 = 0
    Hdh56 = Hdh(theta6, d6, alpha6, a6)     
    
    # ワールド座標系からn番目のlinkまでの同時変換行列を定義
    # ワールド->link2 (=ワールド->link1->link2)
    Hdh012     = Hdh01     @ Hdh12 
    # ワールド->link3 (=ワールド->link1->link2->link3)
    Hdh0123    = Hdh012    @ Hdh23 
    # ワールド->link4 (=ワールド->link1->link2->link3->link4)
    Hdh01234   = Hdh0123   @ Hdh34 
    # ワールド->link5 (=ワールド->link1->link2->link3->link4->リンク5)
    Hdh012345  = Hdh01234  @ Hdh45 
    # ワールド->link6 (=ワールド->link1->link2->link3->link4->リンク5->リンク6)
    Hdh0123456 = Hdh012345 @ Hdh56 
    
    # 各linkの原点座標
    o0 = [             0.0,              0.0,              0.0] # ワールド座標
    o1 = [     Hdh01[0][3],      Hdh01[1][3],      Hdh01[2][3]] # link1の原点座標
    o2 = [    Hdh012[0][3],     Hdh012[1][3],     Hdh012[2][3]] # link2の原点座標
    o3 = [   Hdh0123[0][3],    Hdh0123[1][3],    Hdh0123[2][3]] # link3の原点座標
    o4 = [  Hdh01234[0][3],   Hdh01234[1][3],   Hdh01234[2][3]] # link4の原点座標
    o5 = [ Hdh012345[0][3],  Hdh012345[1][3],  Hdh012345[2][3]] # link5の原点座標
    o6 = [Hdh0123456[0][3], Hdh0123456[1][3], Hdh0123456[2][3]] # link6の原点座標
    # ここまでDH法の実装 -----------------------------------------------------


    # 以下，描画関連 ---------------------------------
    # 各linkの描画（dh法に基づいたlinkなので，長さが0になるlinkが出現する可能性があることに注意）
    # ax.plot([n-1番目のlinkの原点座標(x方向), n番目のlinkの原点座標（x方向）], [n-1番目のlinkの原点座標(y方向), n番目のlinkの原点座標（y方向）], [n-1番目のlinkの原点座標(z方向), n番目のlinkの原点座標（z方向）])
    ax1.plot([o0[0], o1[0]], [o0[1], o1[1]], [o0[2], o1[2]], "-", color="tomato", ms=6) # link1
    ax1.plot([o1[0], o2[0]], [o1[1], o2[1]], [o1[2], o2[2]], "-", color="lightgreen", ms=6) # link2
    ax1.plot([o2[0], o3[0]], [o2[1], o3[1]], [o2[2], o3[2]], "-", color="cyan", ms=6) # link3
    ax1.plot([o3[0], o4[0]], [o3[1], o4[1]], [o3[2], o4[2]], "-", color="purple", ms=6) # link4
    ax1.plot([o4[0], o5[0]], [o4[1], o5[1]], [o4[2], o5[2]], "-", color="gold", ms=6) # link5
    ax1.plot([o5[0], o6[0]], [o5[1], o6[1]], [o5[2], o6[2]], "-", color="mediumslateblue", ms=6) # link6
    
    # 各linkの座標軸を描画
    draw_link_coordinate(ax1,      Hdh01)
    draw_link_coordinate(ax1,     Hdh012)
    draw_link_coordinate(ax1,    Hdh0123)
    draw_link_coordinate(ax1,   Hdh01234)
    draw_link_coordinate(ax1,  Hdh012345)
    draw_link_coordinate(ax1, Hdh0123456)
    
    # 範囲設定
    ax1.set_xlim(-5, 5)
    ax1.set_ylim(-5, 5)
    ax1.set_zlim(-5, 5)

    # 軸ラベル
    ax1.set_xlabel('x')
    ax1.set_ylabel('y')
    ax1.set_zlabel('z')

    plt.show()
draw(6)

<function __main__.link_slider_and_text(slider_widgets_name, text_widgets_name, num)>